In [ ]:
%load_ext autoreload
%autoreload 2

import os

import soundfile
import numpy as np
import wandb
api = wandb.Api()

from thesis.notebook_util import audio_bytes_to_np, play_audio, specplot
from thesis.timbre_transfer_util import get_lufs

In [ ]:
runs = api.runs('neural-audio-synthesis-thesis/nas-evaluation', {
    "$and": [{
    'created_at': {
        "$lt": '2022-08-31T##',
        "$gt": '2022-08-01T##'
        }
    }]
})

In [ ]:
runs.length

In [ ]:
run1 = None
run2 = None

for r in runs:
    # print(r.name, r.id)
    if run1 is None and r.name.endswith("0805-ddspae-cnn-rt"):
        run1 = r

    if run2 is None and r.name.endswith("0805-eval-0804-ddspae-cnn-3-rtq"):
        run2 = r

print(run1.name, run1.id)
print(run2.name, run2.id)

In [ ]:
SAMPLES_DIR = "/Users/vaclav/prog/thesis/data/wandb-samples"


def download_audio_samples(run, samples_dir=SAMPLES_DIR, name_must_contain="audio_both"):
    paths = []
    for file in run.files():
        if file.mimetype.startswith("audio"):
            if name_must_contain in file.name:
                f = file.download(samples_dir, replace=True)
                paths.append(f.name)
                f.close()

    return paths

In [ ]:
paths1 = download_audio_samples(run1)
paths2 = download_audio_samples(run1)

In [ ]:

def prepare_samples(paths_per_model):
    for model in paths_per_model:
        assert len(model["paths"]) == len(paths_per_model[0]["paths"]), f"Different number of samples in {model['name']} {model['id']}"

    out_dir = os.path.join(SAMPLES_DIR, "preprocessed")

    for j, cur_paths in enumerate(zip(*[p["paths"] for p in paths_per_model])):
        # audios = [[] for _ in range(len(paths_per_model) + 1)]
        audio_length = None
        audio_gt = None

        for i, path in enumerate(cur_paths):
            with open(path, "rb") as f:
                audio_cur = audio_bytes_to_np(f.read(), normalize_db=None)

                if not audio_length:
                    audio_length = len(audio_cur)

                audio_pred_cur = audio_cur[:audio_length // 2]
                audio_gt_cur = audio_cur[audio_length // 2:]

                if audio_gt is None:
                    audio_gt = audio_gt_cur
                    # audios[-1].append(audio_gt)

                    output_path = os.path.join(out_dir, f"sample_{j}_gt.wav")
                    soundfile.write(output_path, audio_gt_cur, samplerate=16000)
                else:
                    assert np.allclose(audio_gt_cur, audio_gt)

                # audios[i].append(audio_pred_cur)

            output_path = os.path.join(out_dir, f"sample_{j}_{paths_per_model[i]['name']}.wav")
            soundfile.write(output_path, audio_pred_cur, samplerate=16000)

        # audio = np.clip(audio, -1, 1)
        # audio_
        # lufs = get_lufs(audio)
        # print(lufs)
        # if lufs >= -45:
        # play_audio(audio)

paths_per_model = [
    {"name": run1.name, "id": run1.id, "paths": paths1},
    {"name": run2.name, "id": run2.id, "paths": paths2},
]

prepare_samples(paths_per_model)

In [ ]:
samples_dir = "/Users/vaclav/prog/thesis/data/wandb-samples"

paths = []
for file in run.files():
    if file.mimetype.startswith("audio"):
        if "audio_both" in file.name:
            f = file.download(samples_dir, replace=True)
            paths.append(f.name)
            f.close()

            with open(paths[-1], "rb") as f:
                audio = audio_bytes_to_np(f.read())

            audio = np.clip(audio, -1, 1)
            # audio_
            # lufs = get_lufs(audio)
            # print(lufs)
            # if lufs >= -45:
            play_audio(audio)
            break

In [ ]:
paths

In [ ]:
with open(paths[0], "rb") as f:
    audio = audio_bytes_to_np(f.read())

In [ ]:
get_lufs(audio)

In [ ]:
play_audio(audio)

In [ ]:
"0805-eval-0804-ddspae-cnn-4-rt 3v5tsaqh"


In [ ]:
runs.more